# Cleaning Data using R
By Shuhei Kitamura

### Outline
Research Question: Is there any relationship between Election-Day temperature and electoral outcomes?

Let's clean two types of datasets (Senate [election results](https://transition.fec.gov/pubrec/electionresults.shtml) and [daily temperature](https://aqs.epa.gov/aqsweb/airdata/download_files.html)).

1. Importing Data
2. Treating Missing Values
3. Keeping Columns
4. Keeping Rows
5. Treating Other Values
6. Saving Data

In [12]:
library(readxl) #install.packages('readxl',repos='http://cran.us.r-project.org')

In [13]:
options(repr.matrix.max.rows=200, repr.matrix.max.cols=100) # set # of rows and columns to display 

In [14]:
setwd('...') # set the working directory 

## 1. Importing Data
- For csv files, use `read.table()`.
    - Be sure to include `sep=','`.
    - By adding `header=TRUE`, R recognizes that the first row is column names. 
    - By adding `stringsAsFactors=FALSE`, R does not change characters to factors.
- For tsv files, use `read.table()`.
- For excel files, use `read_excel()` in `readxl` package.
    - Alternatively, use `read.xlsx()` in `xlsx` package or `readWorksheetFromFile()` in `XLConnect` package. Both packages seem to require 64-bit Java for Windows users.
- Alternatively, you could use `fread` in `data.table` package for large csv or tsv data.
- Datasets
    - `elec_senate.xlsx`: US Senate general election results 2008-2014
    - `daily_TEMP_XXXX.csv`: US temperature 2008-2014

In [16]:
elec_data <- list()
temp_data <- list()
for (year in seq(2008,2014,by=2)) {
    elec_data[[paste0('elec_',year)]] <- as.data.frame(read_excel('data/elec_senate.xlsx', sheet=as.character(year)))
    temp_data[[paste0('elec_',year)]] <- as.data.frame(read.table(paste0('data/daily_TEMP_', year, '.csv'), header=TRUE, sep=',', stringsAsFactors=FALSE))     
}

### Checking column names and data entries
- What is the type of `elec_data` and `elec_data[['elec_2008']]`?
    - What happens if you use `elec_data['elec_2008']` instead?
- Print the list of column names in `elec_data[['elec_2008']]`.
- To see some samples of the data, use `head(,#)` or `tail(,#)`, where `#` means the number of rows. 
- What is the type of each column?

### Checking the key
- Next, check whether data have a unique and non-missing key.
- Does `elec_data[['elec_2008']]` have such a key?
    - To check the uniqueness, use `any(duplicated(mydata$column))`.
    - To check the non-missingness, use `any(is.na(mydata$column))`.
- Do the same thing for the remaining years and temperature data later.

## 2. Treating Missing Values
- Print `elec_data[['elec_2008']]`. Why some values are missing?
- There are several strategies to handle missing data:
    - 1. Use a mask
    - 2. Replace it with a sentinel value (e.g., -999)
    - 3. Do nothing (decide later)

In [ ]:
elec_data[['elec_2008']]

### What are the missing values in R? 
- There are three types.
    - `NULL`: Non existence.
    - `NaN` (Not a Number): A missing numerical value (double).
    - `NA` (Not Available): A missing value (logical).
    - (`Inf` is an infinite number.)
- Print vectors in the below example.
- What happens if you aggregate all items in each vector?
    - Try `sum()` and `sum( , na.rm=TRUE)`.

In [40]:
vec1 <- c(1, 2, 3, NA)
vec2 <- c(1, 2, 3, NaN)
vec3 <- c(1, 2, 3, NULL)
vec4 <- c(1, 2, 3, Inf)

### Handling Missing Values
- Useful methods
    - Checking: `is.na()` and `!is.na()`.
- Also, you may combine them with `any()`.
- Replace missing values with zero in `vec1` or `df1`.
    - Use `myvec[is.na(myvec)] <- 0`.
    - Use `mydf[is.na(mydf)] <- 0`.
- Drop missing values in `vec1` or `df1`.
    - Use `myvec <- myvec[!is.na(myvec)]`.
    - Use `mydf <- mydf[!is.na(mydf)]`.

In [52]:
vec1 <- c(1.0, NaN, 3.0, NA, NULL)
df1 <- as.data.frame(matrix(c(1.0, NA, 3.0, NA, 5.0, NaN, 7.0, 8.0, NaN, NaN, NaN, NaN, NA, NA, NA, NA), ncol=4))

- To drop columns whose rows are all missing, use `colSums`.
- To drop rows whose rows are all missing, use `rowSums`.

In [ ]:
df1 <- as.data.frame(matrix(c(1.0, NA, 3.0, NA, 5.0, NaN, 7.0, 8.0, NaN, NaN, NaN, NaN, NA, NA, NA, NA), ncol=4))
print(df1)
print(colSums(is.na(df1)))
print(rowSums(is.na(df1)))
df2 <- df1[,colSums(is.na(df1)) != nrow(df1)]
df3 <- df1[rowSums(is.na(df1)) != ncol(df1),]

## 3. Keeping Columns
- Data may contain some redundant columns that will never be used in analysis. We will drop such columns to reduce the data size.
- Possible candidates:
    1. A column whose values are all missing
    2. A column whose information is not important
- You should be very careful about deciding which columns to keep/drop.
    - You may not use those columns in the current project but may use them in another project.
- Let's check `elec_data[['elec_2008']]`.

In [ ]:
elec_data[['elec_2008']]

- All columns look good. What about `temp_data`?

In [ ]:
head(temp_data[['elec_2008']])
unique(temp_data[['elec_2008']]['Parameter.Name'])

- Let's drop `Parameter.Name`, `Sample.Duration`, `Pollutant.Standard`, `Units.of.Measure`, and `AQI`.
- First of all, let's drop columns that are all missing.

In [17]:
for (year in seq(2008,2014,by=2)) {
    temp_data[[paste0('elec_', year, '_keep')]] <- temp_data[[paste0('elec_', year)]][, colSums(is.na(temp_data[[paste0('elec_',year)]])) != nrow(temp_data[[paste0('elec_',year)]])]    
}

- Next, drop remaining columns.

In [18]:
for (year in seq(2008,2014,by=2)) {
    temp_data[[paste0('elec_',year, '_keep')]] <- temp_data[[paste0('elec_', year, '_keep')]][, !names(temp_data[[paste0('elec_', year, '_keep')]]) %in% c('Parameter.Name', 'Sample.Duration', 'Units.of.Measure')]
}

### Changing Column Names
- Once we have all columns we need, it is time to modify column names. 
- If column names are very long, use uppercase, have space(s), or written in any other non-generic format, we need to change them.
- To simplify the process, we just replace periods with underscores and change uppercase to lowercase.
    - You could use `gsub` and `tolower`.
    - Some column names may still be too long.

In [ ]:
colnames(temp_data[['elec_2008_keep']])
for (year in seq(2008,2014,by=2)) {
    names(temp_data[[paste0('elec_',year, '_keep')]]) <- gsub("\\.", "_", names(temp_data[[paste0('elec_',year, '_keep')]]))
    names(temp_data[[paste0('elec_',year, '_keep')]]) <- tolower(names(temp_data[[paste0('elec_',year, '_keep')]]))
}
colnames(temp_data[['elec_2008_keep']])

## 4. Keeping Rows
- Since columns are all good by now, we move on to rows.
- We need to decide how much rows to keep in the final data.
- Things to consider:
    - Should we keep rows that are all missing?
    - If there are multiple entries per unit of observation, should we keep all of them?

- First, let's look at `elec_data[['elec_2008']]` again. Recall that all election results are missing for some rows.

In [ ]:
elec_data[['elec_2008']]

- Let's drop such rows.

In [ ]:
for (year in seq(2008,2014,by=2)) {
    elec_data[[paste0('elec_', year, '_keep')]] <- elec_data[[paste0('elec_', year)]][rowSums(is.na(elec_data[[paste0('elec_', year)]][c('gelec_dem', 'gelec_rep', 'gelec_oth')])) != ncol(elec_data[[paste0('elec_', year)]][c('gelec_dem', 'gelec_rep', 'gelec_oth')]), ]
}
elec_data[['elec_2008_keep']]

- Next, print `date_local` column in `temp_data[['elec_2008']]` for New York.
    - Should we drop all dates that are not needed for the analysis? Election Days are:
        - November 4th, 2008, November 2nd, 2010, November 6th, 2012, November 8th, 2014

In [ ]:
temp_data[['elec_2008_keep']][temp_data[['elec_2008_keep']]['state_name'] == 'New York','date_local']

- I suggest we keep all dates. 
    - The reason: We may use daily temperature on non-Election days as well.
- Next, print `temp_data[['elec_2008_keep']]` for New York and `2008-11-04`, i.e., Election Day.
    - Are there multiple entries? Why so?

In [ ]:
temp_data[['elec_2008_keep']][(temp_data[['elec_2008_keep']]['state_name'] == 'New York') & (temp_data[['elec_2008_keep']]['date_local'] == '2008-11-04'),]

- What should we do? There are several options. What are the pros and cons of each method?
    - 1. Aggregate data
    - 2. Reshape data
    - 3. Drop some observations
    - 4. Do nothing (decide later)

- There is no right answer!
- Let's take the fourth option for now. Two reasons: 
    - There is no point for losing rich information in the data (regarding 1 and 3). 
    - There will be many missing values if we reshape them (regarding 2).

## 5. Treating Other Values
- It could often be the case that you need to modify entries in a dataset "by hand".
    - Example: some numeric values are written in strings.
- The bottom line: It is better to keep code for whatever you do for original data, if possible.

## 6. Saving Data
- You can save data in any format. However, the most preferred one is csv. Why?
    - Since csv is just a text file, you can read it in any text editor. Easy to share.
    - By contrast, if you do not want to lose information about formatting, macros, etc., it may be better to save it in excel format.
- To save data as a csv file, use `write.table()`.
    - `sep=","` option should be added for csv files.
    - `na=""` means that missing values will be produced as blank cells.
    - `row.names=FALSE` means that row names will not be written as a character vector of row names.
- Use `write.xlsx()` in `readxl` package to save in excel format.

In [21]:
for (year in seq(2008,2014,by=2)) {
    write.table(elec_data[[paste0('elec_', year, '_keep')]], file=paste0('data/elec_senate_R_', year, '.csv'), sep=",", na="", row.names=FALSE)
    write.table(temp_data[[paste0('elec_', year, '_keep')]], file=paste0('data/daily_temp_R_', year, '.csv'), sep=",", na="", row.names=FALSE)   
}